In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications import EfficientNetB3
import math
from sklearn.neighbors import NearestNeighbors
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
!pip install efficientnet -q
import efficientnet.tfkeras as efn

In [ ]:
GET_CV = True
df = pd.read_csv('../input/shopee-product-matching/test.csv')
if len(df) > 3:
    GET_CV = False
del df

In [ ]:
if GET_CV:
    df = pd.read_csv('../input/shopee-product-matching/train.csv')
    image_paths = '../input/shopee-product-matching/train_images/' + df['image']
else:
    df = pd.read_csv('../input/shopee-product-matching/test.csv')
    image_paths = '../input/shopee-product-matching/test_images/' + df['image']

In [ ]:
num_classes = 11014
batch_size = 32

In [ ]:
def get_lr_callback():
    lr_start   = 0.000001
    lr_max     = 0.000005 * batch_size
    lr_min     = 0.000001
    lr_ramp_ep = 5
    lr_sus_ep  = 0
    lr_decay   = 0.8
   
    def lrfn(epoch):
        if epoch < lr_ramp_ep:
            lr = (lr_max - lr_start) / lr_ramp_ep * epoch + lr_start   
        elif epoch < lr_ramp_ep + lr_sus_ep:
            lr = lr_max    
        else:
            lr = (lr_max - lr_min) * lr_decay**(epoch - lr_ramp_ep - lr_sus_ep) + lr_min    
        return lr

    lr_callback = tf.keras.callbacks.LearningRateScheduler(lrfn, verbose = True)
    return lr_callback

In [ ]:
class ArcMarginProduct(tf.keras.layers.Layer):
    def __init__(self, n_classes, s=30, m=0.50, easy_margin=False,
                 ls_eps=0.0, **kwargs):

        super(ArcMarginProduct, self).__init__(**kwargs)

        self.n_classes = n_classes
        self.s = s
        self.m = m
        self.ls_eps = ls_eps
        self.easy_margin = easy_margin
        self.cos_m = tf.math.cos(m)
        self.sin_m = tf.math.sin(m)
        self.th = tf.math.cos(math.pi - m)
        self.mm = tf.math.sin(math.pi - m) * m

    def get_config(self):

        config = super().get_config().copy()
        config.update({
            'n_classes': self.n_classes,
            's': self.s,
            'm': self.m,
            'ls_eps': self.ls_eps,
            'easy_margin': self.easy_margin,
        })
        return config

    def build(self, input_shape):
        super(ArcMarginProduct, self).build(input_shape[0])

        self.W = self.add_weight(
            name='W',
            shape=(int(input_shape[0][-1]), self.n_classes),
            initializer='glorot_uniform',
            dtype='float32',
            trainable=True,
            regularizer=None)

    def call(self, inputs):
        X, y = inputs
        y = tf.cast(y, dtype=tf.int32)
        cosine = tf.matmul(
            tf.math.l2_normalize(X, axis=1),
            tf.math.l2_normalize(self.W, axis=0)
        )
        sine = tf.math.sqrt(1.0 - tf.math.pow(cosine, 2))
        phi = cosine * self.cos_m - sine * self.sin_m
        if self.easy_margin:
            phi = tf.where(cosine > 0, phi, cosine)
        else:
            phi = tf.where(cosine > self.th, phi, cosine - self.mm)
        one_hot = tf.cast(
            tf.one_hot(y, depth=self.n_classes),
            dtype=cosine.dtype
        )
        if self.ls_eps > 0:
            one_hot = (1 - self.ls_eps) * one_hot + self.ls_eps / self.n_classes

        output = (one_hot * phi) + ((1.0 - one_hot) * cosine)
        output *= self.s
        return output

In [ ]:
def get_image_embeddings(image_paths):
    embeds = []
    
    margin = ArcMarginProduct(
                n_classes = num_classes, 
                s = 30, 
                m = 0.5, 
                name='head/arc_margin', 
                dtype='float32'
                )
    inp = tf.keras.layers.Input(shape = (512, 512, 3), name = 'inp1')
    label = tf.keras.layers.Input(shape = (), name = 'inp2')
    #x = EfficientNetB3(weights='imagenet', include_top = False)(inp)
    x = efn.EfficientNetB3(weights='imagenet', include_top=False)(inp)
    x = tf.keras.layers.GlobalAveragePooling2D()(x)
    x = margin([x, label])
        
    output = tf.keras.layers.Softmax(dtype='float32')(x)

    model = tf.keras.models.Model(inputs = [inp, label], outputs = [output])
    model.load_weights('../input/effweigths/effnetb3_epf020_loss0.852.h5')

    model = tf.keras.models.Model(inputs = model.input[0], outputs = model.layers[-4].output)
    chunk = 5000
    iterator = np.arange(np.ceil(len(df) / chunk))
    for j in iterator:
        a = int(j * chunk)
        b = int((j + 1) * chunk)
        image_dataset = get_dataset(image_paths[a:b])
        image_embeddings = model.predict(image_dataset)
        embeds.append(image_embeddings)
    del model
    image_embeddings = np.concatenate(embeds)
    print(f'Our image embeddings shape is {image_embeddings.shape}')
    del embeds
    #gc.collect()
    return image_embeddings


In [ ]:
def decode_image(image_data):
    image = tf.image.decode_jpeg(image_data, channels = 3)
    image = tf.image.resize(image, (512, 512))
    image = tf.cast(image, tf.float32) / 255.0
    return image

In [ ]:
def read_image(image):
    image = tf.io.read_file(image)
    image = decode_image(image)
    return image

In [ ]:
AUTO = tf.data.experimental.AUTOTUNE
def get_dataset(image):
    dataset = tf.data.Dataset.from_tensor_slices(image)
    dataset = dataset.map(read_image, num_parallel_calls = AUTO)
    dataset = dataset.batch(32)
    dataset = dataset.prefetch(AUTO)
    return dataset

In [ ]:
def get_text_embeddings(df, max_features = 15500):
    model = TfidfVectorizer(stop_words = 'english', binary = True, max_features = max_features)
    text_embeddings = model.fit_transform(df['title'])
    print(f'Our title text embedding shape is {text_embeddings.shape}')
    del model
    return text_embeddings

In [ ]:
image_embeddings = get_image_embeddings(image_paths)
text_embeddings = get_text_embeddings(df, max_features = 15500)

In [ ]:
def get_neighbors(df, image_embeddings, text_embeddings, KNN = 100):
    neighbors_model = NearestNeighbors(n_neighbors = KNN, metric = 'cosine').fit(image_embeddings)
    image_distances, image_indices = neighbors_model.kneighbors(image_embeddings)
    neighbors_model = NearestNeighbors(n_neighbors = KNN, metric = 'cosine').fit(text_embeddings)
    text_distances, text_indices = neighbors_model.kneighbors(text_embeddings)
  
    if GET_CV:
        predictions = []
        for k in range(df.shape[0]):
            idx_image = np.where(image_distances[k,] < 0.46)[0]
            ids_image = image_indices[k,idx_image]
            idx_text = np.where(text_distances[k,] < 0.30)[0]
            ids_text = text_indices[k,idx_text]
            ids = list(set(list(ids_image) + list(ids_text)))
            posting_ids = ' '.join(df['posting_id'].iloc[ids].values)
            predictions.append(posting_ids)
    
    else:
        predictions = []
        for k in range(df.shape[0]):
            idx_image = np.where(image_distances[k,] < 0.37)[0]
            ids_image = image_indices[k,idx_image]
            idx_text = np.where(text_distances[k,] < 0.21)[0]
            ids_text = text_indices[k,idx_text]
            ids = list(set(list(ids_image) + list(ids_text)))
            posting_ids = ' '.join(df['posting_id'].iloc[ids].values)
            predictions.append(posting_ids)
        
    return df, predictions


In [ ]:
df, predictions = get_neighbors(df, image_embeddings, text_embeddings, KNN = 50)
df['matches'] = predictions

In [ ]:
df

In [ ]:
df[['posting_id', 'matches']].to_csv('submission.csv', index = False)